<a href="https://colab.research.google.com/github/tlsgptj/MLP/blob/MLP1/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%95%9C%EA%B0%95%EB%8C%80%EA%B5%90(MLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/02. 서울시_한강대교.csv", encoding="cp949")

In [ ]:
df.head(10)

In [ ]:
df.tail(10)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
numeric_values = pd.to_numeric(df['유량(cms)'], errors='coerce')
df['유량(cms)'] = numeric_values.fillna(numeric_values.mean())

In [ ]:
#유량의 결측치를 대체하기 위해 유량의 평균 값으로 NaN를 변경해준다.
df.fillna({'유량(cms)':float(df['유량(cms)'].mean())},inplace=True)

In [ ]:
#지점과 지점명의 경우 모두 같은 데이터이므로 지워준다.
df = df.drop(columns=["지점", "지점명"])
#columns가 평균기온 일강수량 상대습도 유량 4가지 항목으로 정리되었다.

In [ ]:
#데이터 간의 관계를 파악하기 위해 matplotlib 라이브러리를 불러온다.
import matplotlib.pyplot as plt

In [ ]:
import matplotlib as mpl

In [ ]:
ax = df.plot(kind='kde', title='습도에 따른 유량 그래프', figsize=(12,4), legend=True, fontsize=12)
ax.set_xlabel('유량(cms)', fontsize=12)
ax.set_ylabel('평균 상대습도(%)', fontsize=12)
#두 데이터의 단위가 너무 차이가 많이 나 정확한 결과를 알수 없다. 따라서 정규화필요

In [ ]:
df["flow_normalization"] = df["유량(cms)"].astype(float)/abs(df["유량(cms)"].astype(float).max())
df[["유량(cms)", "flow_normalization"]].head()

In [ ]:
#강수량 정규화
df["rain_normalization"] = df["일강수량(mm)"].astype(float)/abs(df["일강수량(mm)"].astype(float).max())
df[["일강수량(mm)", "rain_normalization"]].head()

In [ ]:
df["humidity_normalization"] = df["평균 상대습도(%)"].astype(float)/abs(df["평균 상대습도(%)"].astype(float).max())
df[["평균 상대습도(%)", "humidity_normalization"]].head()

In [ ]:
#그래프를 다시 이쁘게 그려보장
import seaborn as sns
sns.distplot(df["flow_normalization"])

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(df["flow_normalization"], df["humidity_normalization"])
plt.xlabel("humidity_normalization")
plt.ylabel("flow_normalization")
plt.title("습도, 유량 그래프")
plt.show()
#습도가 높아질수록 유량도 증가한다는 사실을 도출했다.

In [ ]:
df["temp_normalization"] = df["평균기온(°C)"].astype(float)/abs(df["평균기온(°C)"].astype(float).max())
df[["평균기온(°C)", "temp_normalization"]].head()

In [ ]:
plt.scatter(df["flow_normalization"], df["평균기온(°C)"])
plt.xlabel("flow_normalization")
plt.ylabel("평균기온(°C)")
plt.title("온도, 유량 그래프")
plt.show()

In [ ]:
#강수량과 유량을 비교해보기로 했다...
plt.scatter(df["flow_normalization"], df["일강수량(mm)"])
plt.xlabel("flow_normalization")
plt.ylabel("일강수량(mm)")
plt.title("강수량, 유량 그래프")
plt.show()
#강수량이 증가할수록 유량이 증가하는 추세를 나타내고 있다.

In [ ]:
#MLP를 사용하여 습도에 따른 유량을 분석해보기로 했다.
#주요 라이브러리 컴파일
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
# 시계열 데이터 생성
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
# 데이터프레임 생성
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
# 훈련 데이터와 테스트 데이터 분리
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#adam
#Test loss: 0.0028341880533844233
#1/1 [==============================] - 0s 85ms/step
#Next value prediction: [[0.03575461]

In [ ]:
# 시계열 데이터 생성
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
# 데이터프레임 생성
df_2 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_2['data'].iloc[i:i+window_size].values)
    y.append(df_2['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
# 훈련 데이터와 테스트 데이터 분리
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='SGD', loss='mean_squared_error')

In [ ]:
# 모델 훈련
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_2['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#SDG 500
#Test loss: 0.004261848051100969
#1/1 [==============================] - 0s 66ms/step
#Next value prediction: [[0.02730757]]

In [ ]:
# 시계열 데이터 생성
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_3 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_3['data'].iloc[i:i+window_size].values)
    y.append(df_3['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='AdaDelta', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#AdaDelta
#Test loss: 0.005228295456618071
#1/1 [==============================] - 0s 96ms/step
#Next value prediction: [[0.01332265]]

In [ ]:
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_3 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_3['data'].iloc[i:i+window_size].values)
    y.append(df_3['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Adagrad', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_3['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#Adagrad
#Test loss: 0.004852883517742157
#1/1 [==============================] - 0s 68ms/step
#Next value prediction: [[0.02171839]]

In [ ]:
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_5 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_5['data'].iloc[i:i+window_size].values)
    y.append(df_5['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Nadam', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_3['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#Nadam
#Test loss: 0.002889535389840603
#1/1 [==============================] - 0s 84ms/step
#Next value prediction: [[0.03834122]]

In [ ]:
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_7 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_7['data'].iloc[i:i+window_size].values)
    y.append(df_7['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Adamax', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_7['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#Adamax
#Test loss: 0.002559814602136612
#1/1 [==============================] - 0s 24ms/step
#Next value prediction: [[0.01271451]]

In [ ]:
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_7 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_7['data'].iloc[i:i+window_size].values)
    y.append(df_7['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_7['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#RMSProp
#Test loss: 0.002466383157297969
#1/1 [==============================] - 0s 61ms/step
#Next value prediction: [[0.00796306]]

In [ ]:
#데이터 프레임 형성
#표로 만들어 가장 오차가 적은 것을 분석해보았다.
#분석내용 표로 도출
data = {
    'Optimizer': ['adam', 'SGD', 'Adadelta', 'Adagrad', 'Adamax', 'RMSProp', 'Nadam'],
    '오차': [0.0028341880533844233, 0.004261848051100969, 0.005228295456618071, 0.004852883517742157, 0.002559814602136612, 0.002466383157297969, 0.002889535389840603],
    'Epoch': [150, 150, 150, 150, 150, 150, 150],
    'Next value prediction': [0.03575461, 0.02730757, 0.01332265, 0.02171839, 0.01271451, 0.00796306, 0.03834122]
}

df_7 = pd.DataFrame(data, dtype=float)

In [ ]:
df_7

In [ ]:
#이것을 그래프로 시각화 해보겠다.
x=df_7["Optimizer"]
y1=df_7["오차"]

# 그래프 그리기
plt.plot(x, y1, label='error')

# 그래프 제목, 축 레이블 설정
plt.title('What would be the most appropriate?')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')

# 범례(legend) 표시
plt.legend()

# 그래프 보이기
plt.show()

#그래프를 통해서 RMSProp가 가장 작은 오차를 가지고 있다는 것을 알 수 있었다.
#가장 큰 오차는 Adadelta였다.

In [ ]:
#RMSProp를 사용해 Epoch, Batch_size, keras의 Dense를 바꾸어보며 가장 적합한 모델을 도출한다.

In [ ]:
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_8 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_8['data'].iloc[i:i+window_size].values)
    y.append(df_8['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=200, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_8['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#epochs=200, batch_size=64)
#48/24
#Test loss: 0.0022740799468010664
#1/1 [==============================] - 0s 90ms/step
#Next value prediction: [[0.01652841]]

In [ ]:
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_8 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_8['data'].iloc[i:i+window_size].values)
    y.append(df_8['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=200, batch_size=32)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_8['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#epochs=200, batch_size=32)
#48/24
#Test loss: 0.002586443442851305
#1/1 [==============================] - 0s 90ms/step
#Next value prediction: [[0.01607266]]

In [ ]:
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_8 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_8['data'].iloc[i:i+window_size].values)
    y.append(df_8['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=200, batch_size=32)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_8['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#96/48
#epochs=200, batch_size=32)
#Test loss: 0.0028256792575120926
#1/1 [==============================] - 0s 100ms/step
#Next value prediction: [[0.00406663]]

In [ ]:
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_8 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_8['data'].iloc[i:i+window_size].values)
    y.append(df_8['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=200, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_8['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#epochs=200, batch_size=64)
#96/48
#Test loss: 0.0021875398233532906
#1/1 [==============================] - 0s 65ms/step
#Next value prediction: [[0.01277857]]

In [ ]:
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_8 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_8['data'].iloc[i:i+window_size].values)
    y.append(df_8['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=500, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_8['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#epochs=500, batch_size=64)
#48/24
#Test loss: 0.001968736294656992
#1/1 [==============================] - 0s 102ms/step
#Next value prediction: [[0.01458428]]

In [ ]:
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_8 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_8['data'].iloc[i:i+window_size].values)
    y.append(df_8['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=500, batch_size=32)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_8['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#epochs=500, batch_size=32)
#48/24
#Test loss: 0.0023632191587239504
#1/1 [==============================] - 0s 59ms/step
#Next value prediction: [[0.00542768]]

In [ ]:
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_8 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_8['data'].iloc[i:i+window_size].values)
    y.append(df_8['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=500, batch_size=32)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_8['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#epochs=500, batch_size=32)
#96/48
#Test loss: 0.0017702518962323666
#1/1 [==============================] - 0s 69ms/step
#Next value prediction: [[0.01042784]]

In [ ]:
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_8 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_8['data'].iloc[i:i+window_size].values)
    y.append(df_8['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=500, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_8['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#epochs=500, batch_size=64)
#96/48
#Test loss: 0.0020167178008705378
#1/1 [==============================] - 0s 23ms/step
#Next value prediction: [[0.01011825]]

In [ ]:
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_8 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_8['data'].iloc[i:i+window_size].values)
    y.append(df_8['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=1000, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_8['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#epochs=1000, batch_size=64)
#96/48
#Test loss: 0.0020732267294079065
#1/1 [==============================] - 0s 64ms/step
#Next value prediction: [[0.01116662]]

In [ ]:
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_8 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_8['data'].iloc[i:i+window_size].values)
    y.append(df_8['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=1000, batch_size=32)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_8['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#epochs=1000, batch_size=32)
#96/48
#Test loss: 0.0021327785216271877
#1/1 [==============================] - 0s 65ms/step
#Next value prediction: [[0.00376695]]

In [ ]:
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_8 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_8['data'].iloc[i:i+window_size].values)
    y.append(df_8['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=1000, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_8['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#epochs=1000, batch_size=64)
#96/48
#Test loss: 0.0018469428177922964
#1/1 [==============================] - 0s 93ms/step
#Next value prediction: [[0.00898385]]

In [ ]:
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_8 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_8['data'].iloc[i:i+window_size].values)
    y.append(df_8['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=1000, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_8['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))

print('Next value prediction:', next_value)

In [ ]:
#48/24
#epochs=1000, batch_size=64)
#Test loss: 0.0017275323625653982
#1/1 [==============================] - 0s 69ms/step
#Next value prediction: [[0.00827242]]

In [ ]:
data = df["humidity_normalization"]
target = df["flow_normalization"]

In [ ]:
df_8 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_8['data'].iloc[i:i+window_size].values)
    y.append(df_8['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=1000, batch_size=32)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_8['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))

print('Next value prediction:', next_value)

In [ ]:
#48/24
#epochs=1000, batch_size=32)
#Test loss: 0.002041981788352132
#1/1 [==============================] - 0s 76ms/step
#Next value prediction: [[0.00998862]]

In [ ]:
#표로 정리
data = {
    '번호' : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'Keras의 Dense 층': ['48/24', '48/24', '96/48', '96/48', '48/24', '48/24', '96/48', '96/48', '48/24', '48/24', '96/48', '96/48'],
    '오차': [0.0022740799468010664, 0.002586443442851305, 0.0021875398233532906, 0.0028256792575120926, 0.001968736294656992, 0.0023632191587239504, 0.0017702518962323666, 0.0020167178008705378, 0.0021327785216271877, 0.0018469428177922964, 0.002041981788352132, 0.0017275323625653982],
    'Epoch': [200, 200, 200, 200, 500, 500, 500, 500, 1000, 1000, 1000, 1000],
    'Next value prediction': [0.01652841, 0.01607266, 0.01277857, 0.00406663, 0.01458428, 0.00542768, 0.01042784, 0.01011825, 0.00376695, 0.00898385, 0.00998862, 0.00827242],
    'Batch_size': [64, 32, 64, 32, 64, 32, 32, 64, 32, 64, 32, 64]
}

df_8 = pd.DataFrame(data, dtype=float)

In [ ]:
df_8

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
x = df_8["번호"]
y1 = df_8["오차"]

# 그래프 그리기
plt.plot(x, y1, label='error')

# 그래프 제목, 축 레이블 설정
plt.title('What would be the most appropriate?')
plt.ylabel('Y-axis')

# 그래프 보이기
plt.show()

#Epoch를 많이 써줄수록 오차가 점점 줄어드는 것을 알 수 있다.